Data sources:
- [Vehicles from Traficom](https://tieto.traficom.fi/en/datatraficom/open-data?toggle=Open%20data%20for%20vehicles)
- [Municipalities from Statistics Finland](https://stat.fi/en/luokitukset/kunta/)
- [Geographic from MAPOG](https://gisdata.mapog.com/finland/administrative_boundaries_level8_polygon)

In [1]:
# Municipalities data
from util.data_import import get_municipalities
municipalities = get_municipalities()

display(municipalities)

{'020': 'Akaa',
 '005': 'Alajärvi',
 '009': 'Alavieska',
 '010': 'Alavus',
 '016': 'Asikkala',
 '018': 'Askola',
 '019': 'Aura',
 '035': 'Brändö',
 '043': 'Eckerö',
 '046': 'Enonkoski',
 '047': 'Enontekiö',
 '049': 'Espoo',
 '050': 'Eura',
 '051': 'Eurajoki',
 '052': 'Evijärvi',
 '060': 'Finström',
 '061': 'Forssa',
 '062': 'Föglö',
 '065': 'Geta',
 '069': 'Haapajärvi',
 '071': 'Haapavesi',
 '072': 'Hailuoto',
 '074': 'Halsua',
 '075': 'Hamina',
 '076': 'Hammarland',
 '077': 'Hankasalmi',
 '078': 'Hanko',
 '079': 'Harjavalta',
 '081': 'Hartola',
 '082': 'Hattula',
 '086': 'Hausjärvi',
 '111': 'Heinola',
 '090': 'Heinävesi',
 '091': 'Helsinki',
 '097': 'Hirvensalmi',
 '098': 'Hollola',
 '102': 'Huittinen',
 '103': 'Humppila',
 '105': 'Hyrynsalmi',
 '106': 'Hyvinkää',
 '108': 'Hämeenkyrö',
 '109': 'Hämeenlinna',
 '139': 'Ii',
 '140': 'Iisalmi',
 '142': 'Iitti',
 '143': 'Ikaalinen',
 '145': 'Ilmajoki',
 '146': 'Ilomantsi',
 '153': 'Imatra',
 '148': 'Inari',
 '149': 'Ingå',
 '151': 'Isojok

In [2]:
# Vehicles data
from util.data_import import get_vehicles
vehicles = get_vehicles()

display(vehicles.dtypes)
display(vehicles.shape)
display(vehicles.head(10))

registration_date    object
intro_date           object
color                object
driving_force        object
is_hybrid            object
maker_text           object
municipality         object
odometer             object
dtype: object

(2752452, 8)

,registration_date,intro_date,color,driving_force,is_hybrid,maker_text,municipality,odometer
0,1984-07-09,19840000,1,01,NaN,Ford,740,NaN
1,1990-05-08,19900508,9,01,NaN,Citroen,091,NaN
2,2003-10-02,20031002,6,01,NaN,Honda,837,284104
3,2006-03-17,20060317,Y,01,NaN,Toyota,989,155944
4,2007-01-05,20070105,2,02,NaN,Toyota,694,2692651
5,1996-03-14,19960314,5,01,NaN,Nissan,777,262227
6,2003-07-01,20030701,8,01,NaN,Honda,851,262915
7,2000-03-24,20000324,6,01,NaN,Nissan,250,185968
8,2005-10-25,20051025,8,01,NaN,BMW,755,205732
9,2001-06-06,20010606,Y,01,NaN,Hyundai,140,389645


In [3]:
import copy
import importlib
import util.data_cleaning

importlib.reload(util.data_cleaning)

df = vehicles.copy(deep=True)
mun = copy.deepcopy(municipalities)

(df, mun) = util.data_cleaning.clean(df, mun)

# Check which high count Other labeled makers are missing from mapping
other_makers = df[df["maker"] == "Other"]
maker_counts = other_makers["maker_text"].value_counts()
top_maker_rows = other_makers[other_makers["maker_text"].isin(maker_counts.index)]
top_maker_counts_in_top_rows = top_maker_rows["maker_text"].value_counts().nlargest(10)
display(top_maker_counts_in_top_rows)

df.reset_index(inplace=True, drop=True)

# NA check
na_rows = df[df[df.columns].isna().any(axis=1)]

# Sanity checks
display(df.dtypes)
display(df.shape) # Same size as before
display(f"Min year: {df["registration_year"].min()}", f"Max year: {df["registration_year"].max()}")
display(na_rows) # Should have none

maker_text
mg           1525
adria        1385
capron       1275
pontiac       671
gm daewoo     650
dethleffs     648
buick         626
hymer         612
ds            575
weinsberg     569
Name: count, dtype: int64

color                object
driving_force        object
maker_text           object
municipality         object
odometer              Int32
registration_year     Int16
maker                object
dtype: object

(2752452, 7)

'Min year: 1979'

'Max year: 2024'

,color,driving_force,maker_text,municipality,odometer,registration_year,maker


In [4]:
# Final data
import json
import importlib
import os
import util.data_cleaning
import util.data_validation


importlib.reload(util.data_cleaning)
importlib.reload(util.data_validation)


final = util.data_cleaning.generate(df, mun, "2024-03-31")  # TODO: dynamic date
valid = util.data_validation.validate(final, mun)

if valid:
    path = os.path.join(os.path.join(os.getcwd(), "data.json"))
    file = json.dumps(final, indent=2, ensure_ascii=False)
    if os.path.isfile(path):
        os.remove(path)
    with open(path, "w", encoding="utf-8") as fh:
        fh.write(file)

display(final.keys())
display(final["municipalities"][0])

dict_keys(['date', 'municipalities'])

{'code': '020',
 'name': 'Akaa',
 'countByDrivingForce': {'1': 6000, '2': 2682, '3': 727, '4': 283, '5': 80},
 'countByColor': {'black': 1378,
  'blue': 1386,
  'brown': 707,
  'green': 401,
  'grey': 2132,
  'other': 252,
  'red': 1431,
  'silver': 649,
  'white': 1436},
 'countByRegistrationYear': {'1979': 74,
  '1980': 7,
  '1981': 7,
  '1982': 13,
  '1983': 25,
  '1984': 29,
  '1985': 42,
  '1986': 55,
  '1987': 74,
  '1988': 82,
  '1989': 86,
  '1990': 96,
  '1991': 38,
  '1992': 40,
  '1993': 39,
  '1994': 37,
  '1995': 45,
  '1996': 72,
  '1997': 76,
  '1998': 109,
  '1999': 147,
  '2000': 165,
  '2001': 164,
  '2002': 247,
  '2003': 318,
  '2004': 312,
  '2005': 395,
  '2006': 384,
  '2007': 358,
  '2008': 438,
  '2009': 293,
  '2010': 398,
  '2011': 447,
  '2012': 365,
  '2013': 350,
  '2014': 403,
  '2015': 419,
  '2016': 438,
  '2017': 423,
  '2018': 458,
  '2019': 434,
  '2020': 355,
  '2021': 360,
  '2022': 308,
  '2023': 297,
  '2024': 50},
 'countByMaker': {'Alfa Romeo':